### Feature Engineering
- Text: TF‑IDF pe titlu.
- Numeric:
- number_of_views → log-transform.
- merchant_rating → scalare.
- listing_date → extragem year, month.


In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv("../data/products_modified.csv")

df["views_log"] = np.log1p(df["number_of_views"])
df["year"] = pd.to_datetime(df["listing_date"]).dt.year
df["month"] = pd.to_datetime(df["listing_date"]).dt.month
